## Code for Figure 1 in the ICML 2025 paper
## Recommendations from Sparse Comparison Data: 
## Provably Fast Convergence for Nonconvex Matrix Factorization

In [1]:
import numpy as np
import pandas as pd
import time
import networkx as nx
import matplotlib.pyplot as plt
import scipy.linalg as la
import pickle
import math
import random
from sklearn.preprocessing import StandardScaler


def g(x):
  s = 1 / (1 + np.exp(-x))
  math.ceil(s * 1e5) / 1e5
  return  s # sigmoid function

def g_prime(x):
  sigmoid = g(x)
  return sigmoid * (1 - sigmoid)  # Derivative of sigmoid function


# ============================================================================
def block_identity_matrix(n1, n2):
    # Create I_1 and I_2
    I_1 = np.eye(n1)
    I_2 = np.eye(n2)

    # Create the block matrix [I_1 0; 0 -I_2]
    top_block = np.hstack((I_1, np.zeros((n1, n2))))
    bottom_block = np.hstack((np.zeros((n2, n1)), -I_2))

    # Combine top and bottom blocks
    block_matrix = np.vstack((top_block, bottom_block))

    return block_matrix

# ============================================================================

def generate_sampling_matrix(n_1, n_2):
    A = np.zeros(3)

    i = np.random.randint(0, n_1)
    j = np.random.randint(0, n_2)
    k = np.random.randint(0, n_2)

    while j == k:
      k = np.random.randint(0, n_2)

    A[0] = i
    A[1] = j
    A[2] = k

    return A

# ============================================================================

def generate_data(A_bar, W, X, n_1, n_2, m, J):

  Z = W @ X.T @ J

  Z_1 = np.zeros((n_1,n_1))
  Z_2 = np.zeros((n_2,n_2))
  Z_3 = np.zeros((n_2, n_1))
  z = np.zeros(m)

  top_block = np.hstack((Z_1, Z))  # [0 Z]
  bottom_block = np.hstack((Z_3, Z_2))  # [0 0]
  ZZ_T = np.vstack((top_block, bottom_block))

  for i in range(m):
    [i_i, j_i, k_i] = A_bar[i]
    i_i = int(i_i)
    j_i = int(j_i)
    k_i = int(k_i)
    inner_product = Z[i_i, j_i] - Z[i_i, k_i]
    z[i] = np.random.binomial(1, g(inner_product))

  return z


# ============================================================================
def orthogonal_procrustes(Z, Z_star):
    """
    Solves:
    arg min_{R^T * R = I} ||Z - Z_star * R||^2
    """

    # Compute the SVD of Z^T * Z_star
    U, _, Vt = np.linalg.svd(Z_star.T @ Z)

    # Compute the orthogonal matrix R
    R = U @ Vt
    R[np.abs(R) < 1e-5] = 0

    return R

# ============================================================================
def compute_min_norm(Z, Z_star):
    """
    Computes
    min_{R^T * R = I}||Z - Z_star * R||^2
    """

    # Get the orthogonal matrix R using the orthogonal_procrustes function
    R = orthogonal_procrustes(Z, Z_star)

    # Compute the minimum norm ||Z - Z_star * R||^2
    min_norm = np.linalg.norm(Z - Z_star @ R, 'fro')

    return min_norm

# ============================================================================
def remove_duplicate_rows(array):
    # Convert the list of lists to a NumPy array
    arr = np.array(array)
    # Use np.unique to remove duplicate rows
    unique_arr = np.unique(arr, axis=0)
    return unique_arr

In [2]:
def nabla_L(Z, ZZ_star, A_bar, lam, D):

    n1_n2, r = Z.shape  # Size of Z: (n_1+n_2) x r
    grad = np.zeros_like(Z)  # Initialize the gradient matrix
    ZZ_T = Z @ Z.T  # Compute ZZ^T
    m = len(A_bar)

    for i in range(m):
        Yi = np.zeros_like(Z)

        [i_i, j_i, k_i] = A_bar[i]
        i_i = int(i_i)
        j_i = int(j_i)
        k_i = int(k_i)

        inner_product = ZZ_T[i_i, n_1 + j_i] - ZZ_T[i_i, n_1 + k_i] # Compute the inner product ⟨A_i, ZZ^T⟩
        inner_product_true = ZZ_star[i_i, n_1 + j_i] - ZZ_star[i_i, n_1 + k_i]

        diff = g(inner_product) - g(inner_product_true)

        fraction_term = 1

        Yi[i_i,:] = Z[n_1 + j_i,:] - Z[n_1 + k_i,:]
        Yi[n_1 + j_i,:] = Z[i_i,:]
        Yi[n_1 + k_i,:] = -Z[i_i,:]

        # Compute the gradient contribution for this i
        grad += fraction_term * diff * Yi

    grad /= m

    # Add the regularization term
    grad += 4 * lam * D @ ZZ_T @ D @ Z

    grad[np.abs(grad) < 5e-6] = 0
    return grad

In [17]:
# Defining the parameters
n_1 = 200
n_2 = 300
n1_n2 = n_1 + n_2
r = 3
c_0 = 1/4
kappa = 1.1
delta = 0.05

J = np.eye(n_2) - np.ones((n_2,n_2))/n_2
D = block_identity_matrix(n_1, n_2)

mu = 1.01
# Set the true matrices W^* and X^*
selected_elements = random.sample(range(n_1), int(n_1/mu))
W = np.zeros((n_1, r))
for i in selected_elements:
    W[i] = np.random.randn(1, r)
    W[i] = W[i] / np.linalg.norm(W[i], axis=0)
W_f = np.linalg.norm(W, 'fro')**2

selected_elements = random.sample(range(n_2), int(n_2/mu))
X = np.zeros((n_2, r))
for i in selected_elements:
    X[i] = np.random.randn(1, r)
    X[i] = X[i] / np.linalg.norm(X[i], axis=0)
X_f = np.linalg.norm(X, 'fro')**2

m = int( (c_0* (r * mu * kappa)**2 * (n1_n2) * np.log(n1_n2/delta)) / np.log(10))


In [4]:
Y = W @ X.T @ J

U, S, Vt = np.linalg.svd(Y, full_matrices=True)
sigm = np.sqrt(S[0:r])
U_t = U[:, 0:r] @ np.diag(sigm)
V_t = Vt.T[:, 0:r] @ np.diag(sigm)

Z_star = np.vstack((U_t, V_t))
Z_star[np.abs(Z_star) < 1e-6] = 0

ZZ_star = Z_star @ Z_star.T


In [16]:

aa = sigm[0]
bb = sigm[-1]

for _ in range(r):
  sigm[_] = sigm[_] * kappa * (aa/ bb)


In [12]:

# Initial point
delt = 1
W_0 = np.random.randn(n_1, r)
X_0 = np.random.randn(n_2, r)
Z_o = np.vstack((W_0, J @ X_0))

Z_0 = Z_star + delt * Z_o

# Algorithm parameters
Itr = 1000
rho = (n_1*n_2) / ((280 * mu * r)**2)
lam = 50 / (n_1 * n_2 )
l_0 = np.linalg.norm(Z_0, ord=2)
tr = np.sqrt((2 * mu * r) / min([n_1,n_2])) * l_0


In [20]:

A_bar = [generate_sampling_matrix(n_1, n_2) for _ in range(m)]

Grad_np = np.zeros(Itr)
err_np = np.zeros(Itr)

Grad_wp = np.zeros(Itr)
err_wp = np.zeros(Itr)

# iterations
Z_wp = np.copy(Z_0)
Z_np = np.copy(Z_0)

for t in range(Itr):
  # Compute the gradient
  grad_wp = nabla_L(Z_wp, ZZ_star, A_bar, lam, D)
  Grad_wp[t] = np.linalg.norm(grad_wp, 'fro')

  grad_np = nabla_L(Z_np, ZZ_star, A_bar, lam, D)
  Grad_np[t] = np.linalg.norm(grad_np, 'fro')
  if Grad_np[t] < 1e-5:
    break

  #Gradient Descent
  Z_wp -= rho * grad_wp
  Z_wp[np.abs(Z_wp) < 1e-6] = 0

  Z_np -= rho * grad_np
  Z_np[np.abs(Z_np) < 1e-6] = 0

  err_wp[t] = compute_min_norm( Z_wp, Z_star )
  err_np[t] = compute_min_norm( Z_np, Z_star )

  if t % 100 == 1:
    print(t)
    print(Grad_np[t], err_np[t])

  #Projection Step
  for i in range(n_1+n_2):
   l_i = np.linalg.norm(Z_wp[i], axis=0)
   if l_i > tr:
     Z_wp[i] = (Z_wp[i] / l_i) * tr



In [11]:
import matplotlib.cm as cm

plt.plot(err_np[:-1]/np.sqrt(n_1*n_2), color=cm.viridis(0/10), linestyle='--',linewidth=1)

plt.plot(err_wp[:-1]/np.sqrt(n_1*n_2), label='With Proj.', color='red', linestyle='--',marker='o', markevery=500, markersize=5)
plt.plot(err_np[:-1]/np.sqrt(n_1*n_2)+.0000001, label='Without Proj.', color='blue')

# Add labels and title
plt.xlabel('Iteration')
plt.ylabel('$||\Delta(Z)||_F/\sqrt{n_1n_2}$')
plt.yscale('log')
plt.grid(True)

plt.legend()
# Display the plot
plt.show()